# Lexical search of Pliny, *Epistulae*

> *Add link here to all the caveats...*


### How to use this notebook

1. First, run step 1 (e.g., by selecting the cell labelled **Step 1: load everything** and choosing "Run all below" from the "Cell" menu).  This will be slow, and your mileage may vary depending on how well your connection to different resources on the internet happens to be performing just then.
2. Just below the cell labelled **Step 2: search by example**, fill in between quotation marks an argument to the function `moreLikeThis`.  E.g., to find all forms of `duco` in Pliny, you could edit that cell to read

    `moreLikeThis("duxit")`

Then run the cell (e.g., by selecting it, and choosing "Run cells" from the "Cell" menu).

# Step 2: search by example

In [ ]:
moreLikeThis("duxit")

# Step 1: load everything


In [ ]:
// set up notebook to find repository
val personalRepo = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(personalRepo)

In [ ]:
// ivy imports
import $ivy.`edu.holycross.shot::latincorpus:7.0.0-pr3`

In [ ]:
import edu.holycross.shot.latincorpus._

In [ ]:
val textUrl = "https://raw.githubusercontent.com/LinguaLatina/analysis/master/data/pliny/pliny-latc.cex"
val text = LatinCorpus.fromUrl(textUrl)



In [ ]:
val lewisShortBase = "http://folio2.furman.edu/lewis-short/index.html?urn=urn:cite2:hmt:ls.markdown:"

def moreLikeThis(s: String) = {
    try {
        val lexemes = text.tokenLexemeIndex(s)     
        val labelledLexemes = text.tokenLabelledLexemeIndex(s)  
        val heading = s"<h2><em>${s}</em> matched ${labelledLexemes.size} lexeme(s): " + labelledLexemes.mkString(", ") + "</h2>"
        
        val htmlBody = for (lex <- lexemes) yield{        
          val lsId = lex.replaceFirst("ls.", "")
          val lsUrl =lewisShortBase + lsId
          val cns = text.citableUnits.sequences.filter( cn => cn.matchesLexeme(lex))
          val para = "<p>For " + lex + ", matched <strong>" + cns.size + "</strong> citable passages.</p>" 
          val xref = "<p> See <a href=\"" + lsUrl + "\">Lewis-Short entry</a></p>"
          
          val bullets = for ((cn) <- cns) yield {
            val txt = cn.lexicalText.mkString(" ")
            s"<li>${txt}</li>"
          }
          para + xref + "<ol>" + bullets.mkString("\n") + "</ol>"     
        }
        Html(heading + htmlBody.mkString("<br/>"))
        
        
    } catch {
        case nope: java.util.NoSuchElementException => {
            Html("<p><em>" + s + "</em> does not occur in Pliny's letters.  Please check the form, and try again.</p>")
        }    
        
        
    }

}